# Final Prompt Evaluation - English

Measurement harness for English prompt iterations.
Compares Prompt A (Baseline) vs Prompt B (Experimental).

In [25]:
import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from groq import Groq
from pybars import Compiler

# Load .env
load_dotenv(find_dotenv(usecwd=True))

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY environment variable not set.")

GROQ_MODEL = "llama-3.1-8b-instant"               #"llama-3.3-70b-versatile"
try:
    client = Groq(api_key=GROQ_API_KEY)
    print(f"✅ Setup complete. Using model: {GROQ_MODEL}")
except Exception as e:
    print(f"❌ Error initializing Groq client: {e}")

✅ Setup complete. Using model: llama-3.1-8b-instant


## Paths & Data Loading

In [26]:
try:
    PROJECT_ROOT = Path(__file__).resolve().parents[1]
except NameError:
    # Fallback if __file__ is not defined (e.g. interactive mode)
    PROJECT_ROOT = Path(os.getcwd()).parent

SAMPLES_DIR = PROJECT_ROOT / "data" / "samples"

SAMPLE_CATEGORIES_EN = {
    "en_academic.txt": "Academic",
    "en_medical.txt": "Medical",
    "en_legal.txt": "Legal",
    "en_insurance.txt": "Insurance",
    "en_technical.txt": "Technical",
    "en_government.txt": "Government",
}

def get_all_samples_en() -> list[dict]:
    samples = []
    for filename in sorted(SAMPLE_CATEGORIES_EN.keys()):
        filepath = SAMPLES_DIR / filename
        if filepath.exists():
            text = filepath.read_text(encoding="utf-8").strip()
            samples.append({
                "filename": filename,
                "category": SAMPLE_CATEGORIES_EN[filename],
                "text": text
            })
    return samples

print(f"Found {len(get_all_samples_en())} English samples in {SAMPLES_DIR}")

Found 6 English samples in /Users/alastair/Github/klartext/data/samples


## Prompts (English)

In [27]:
PROMPT_IDENTITY = """# Identity

You are an expert in plain language writing.
You specialise in rewriting text to be accessible 
to people with learning disabilities or low literacy.
"""

PROMPT_INSTRUCTIONS = """# Core Task 

* Rewrite the input text to be extremely simple and easy to understand.
* Keep the same meaning as the original text.

# Constraints

* Do NOT include any introductory or concluding text (e.g., "Here is the simplified text").
* Output ONLY the simplified text.
* Never output any XML/HTML tags or attributes (no <...>, no id=...).

# Structure & Formatting Rules

* Use clear structure.
* Use bullet points for steps, lists, or multiple items. Otherwise prefer short sentences.
* Add blank lines between every paragraph.
"""

EL_RULES = """# Plain Language Rules
# Sentence & Length Rules

* Use very short sentences (maximum 10 words per sentence).
* Break up long sentences.
* Keep subjects and verbs close together.

# Vocabulary & Wording Rules

* Use simple, familiar words. Avoid technical, foreign, or formal terms.
* Explain any uncommon or necessary technical words or abbreviations in parentheses the first time they appear.
* Explain complex ideas or uncommon nouns in parentheses.
* Use positive wording. Avoid negations and never use double negatives.
* Replace abstract nouns with concrete, active verbs.

# Tone & Audience Rules

* Prefer active voice. Avoid passive voice whenever possible.
* Address the reader directly using “you”.
* Use a friendly, neutral tone.
* Avoid bureaucratic, legalistic, or commanding language.

# Consistency Rules

* Remove filler words and unnecessary details. Keep only essential information.
* Use the same words consistently. Do not switch terms for the same thing.
"""

PROMPT_EXAMPLES = """# Examples
# The following are example pairs.
# Learn the style and constraints from them.
# Do NOT copy the XML tags into your output.

<examples>

  <example id="1">
    <original_text>
    Upon arrival at the facility, visitors are required to sign in at the front desk and present valid photo identification.
    </original_text>

    <simplified_text>
    When you arrive:

    * Go to the front desk.
    * Sign in with your name.
    * Show your photo ID.
    </simplified_text>
  </example>

  <example id="2">
    <original_text>
    The medication should be administered twice daily with food to minimize potential gastrointestinal discomfort.
    </original_text>

    <simplified_text>
    Take this medicine two times every day.

    * Eat food when you take it. This helps your stomach feel better.
    </simplified_text>
  </example>

</examples>
"""

# Assemble Prompt A (Baseline)
PROMPT_A_SYSTEM = f"""{PROMPT_IDENTITY}

{PROMPT_INSTRUCTIONS}

{EL_RULES}

{PROMPT_EXAMPLES}"""

PROMPT_USER_TEMPLATE = "Rewrite this text in simple language:\n{{text}}"

# Assemble Prompt B (Experimental)
# ------------------------------------------------------------
# To experiment, redefine any component below before assembly.
# ------------------------------------------------------------

PROMPT_B_IDENTITY = PROMPT_IDENTITY

PROMPT_B_INSTRUCTIONS = """# Core Task 

* Rewrite the input text to be extremely simple and easy to understand.

# Constraints

* Keep the same meaning as the source text. Ensure that meaning is not dropped from any sentences.
* Organise the text by ideas. When an idea is complex, explain the idea using bullet points in simple language.
* You are making the text easier to understand, you are not condensing or reducing the text.
* Do NOT include any introductory or concluding text (e.g., "Here is the simplified text").
* Output ONLY the simplified text.
* Never output any XML/HTML tags or attributes (no <...>, no id=...).

# Structure & Formatting Rules

* Use clear structure.
* Use bullet points for steps, lists, or multiple items. Otherwise prefer short sentences.
* Add blank lines between every paragraph.
"""

PROMPT_B_RULES = """# Plain Language Rules
# Sentence & Length Rules

* Use very short sentences (maximum 10 words per sentence).
* Break up long sentences.
* Keep subjects and verbs close together.

# Vocabulary & Wording Rules

* Use simple, familiar words. Avoid technical, foreign, or formal terms.
* Explain any uncommon or necessary technical words or abbreviations in parentheses the first time they appear.
* Explain complex ideas or uncommon nouns in parentheses.
* Use positive wording. Avoid negations and never use double negatives.
* Replace abstract nouns with concrete, active verbs.

# Tone & Audience Rules

* Prefer active voice. Avoid passive voice whenever possible.
* Maintain conditional language when required.
* Address the reader personally and directly when relevant.
* Use a friendly, neutral tone.
* Avoid bureaucratic, legalistic, or commanding language.

# Consistency Rules

* Remove filler words and unnecessary details. Keep only essential information.
* Do not explain ideas using the same language.
* Use the same words consistently. Do not switch terms for the same thing.
"""

PROMPT_B_EXAMPLES = PROMPT_EXAMPLES

PROMPT_B_SYSTEM = f"""{PROMPT_B_IDENTITY}

{PROMPT_B_INSTRUCTIONS}

{PROMPT_B_RULES}

{PROMPT_B_EXAMPLES}"""

PROMPT_B_USER_TEMPLATE = PROMPT_USER_TEMPLATE

## Inference Functions

In [28]:
def render_prompt(template_source: str, text: str) -> str:
    compiler = Compiler()
    template = compiler.compile(template_source)
    return template({"text": text})

def call_groq(system_prompt: str, user_prompt: str, api_key: str, model: str = GROQ_MODEL, temperature: float = 0.1, max_tokens: int = 2000) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def run_prompt_on_samples_en(system_prompt: str, user_template: str, api_key: str, samples: list[dict] = None) -> pd.DataFrame:
    if samples is None:
        samples = get_all_samples_en()
    
    results = []
    for s in samples:
        try:
            # System prompt is usually static, user prompt needs rendering
            # But we support handlebars in system prompt too if needed, though not used in Baseline A
            rendered_user = render_prompt(user_template, s["text"])
            
            output = call_groq(system_prompt, rendered_user, api_key)
            results.append({
                "filename": s["filename"],
                "category": s["category"],
                "original_text": s["text"],
                "output": output
            })
        except Exception as e:
             results.append({
                "filename": s["filename"],
                "category": s["category"],
                "original_text": s["text"],
                "output": f"[Error: {e}]"
            })
    
    return pd.DataFrame(results)

## Run & Compare (EN)

In [29]:
MAX_SAMPLES = None # Set to int to limit, e.g. 3

samples = get_all_samples_en()
if MAX_SAMPLES is not None:
    samples = samples[:MAX_SAMPLES]

print(f"Running comparison on {len(samples)} samples...")

# Run Prompt A
print("Running Prompt A...")
df_a = run_prompt_on_samples_en(PROMPT_A_SYSTEM, PROMPT_USER_TEMPLATE, GROQ_API_KEY, samples=samples)

# Run Prompt B
print("Running Prompt B...")
df_b = run_prompt_on_samples_en(PROMPT_B_SYSTEM, PROMPT_B_USER_TEMPLATE, GROQ_API_KEY, samples=samples)

# Merge
comparison_df = df_a[["filename", "category", "original_text", "output"]].copy()
comparison_df = comparison_df.rename(columns={"output": "output_a"})
comparison_df["output_b"] = df_b["output"]

# Display DataFrame
# (If running in VS Code or Jupyter, this renders the table)
try:
    from IPython.display import display
    display(comparison_df)
except ImportError:
    print(comparison_df)

# Print Logic for easy reading
print("\n" + "="*80)
print("DETAILED COMPARISON")
print("="*80)

for index, row in comparison_df.iterrows():
    print(f"\n📂 File: {row['filename']} ({row['category']})")
    print("-" * 40)
    print("ORIGINAL:")
    # Truncate original for cleaner display if very long
    print(row['original_text'][:200] + "..." if len(row['original_text']) > 200 else row['original_text'])
    print("-" * 40)
    print("PROMPT A OUTPUT:")
    print(row['output_a'])
    print("-" * 40)
    print("PROMPT B OUTPUT:")
    print(row['output_b'])
    print("="*80)

Running comparison on 6 samples...
Running Prompt A...
Running Prompt B...


,filename,category,original_text,output_a,output_b
0,en_academic.txt,Academic,The implementation of sustainable development ...,"To achieve sustainable development goals, you ...",We need to understand how different things wor...
1,en_government.txt,Government,Pursuant to the provisions of Section 504 of t...,"When we get money from the government, we must...",We must treat people with disabilities fairly....
2,en_insurance.txt,Insurance,Coverage under this policy shall be contingent...,"To get benefits, you must follow the rules of ...",To get benefits from this policy:\n\n* You mus...
3,en_legal.txt,Legal,The obligations contained herein shall remain ...,You must keep the secrets. \n\n* The secrets w...,These rules will stay in place forever. They w...
4,en_medical.txt,Medical,Patients presenting with symptoms indicative o...,"If you have a heart attack, do this:\n\n* Take...",People who have heart attack symptoms should g...
5,en_technical.txt,Technical,"Prior to initializing the system, ensure that ...",Before you start using the system:\n\n* Make s...,Before you start using the system:\n\n* Make s...



DETAILED COMPARISON

📂 File: en_academic.txt (Academic)
----------------------------------------
ORIGINAL:
The implementation of sustainable development goals necessitates a comprehensive understanding of the multifaceted interdependencies between economic growth, environmental conservation, and social equ...
----------------------------------------
PROMPT A OUTPUT:
To achieve sustainable development goals, you need to understand how:

* The economy
* The environment
* Society

are connected.

This means making big changes in many areas, such as:

* Energy production
* Transportation
* Farming practices

These changes will help create a more sustainable world.
----------------------------------------
PROMPT B OUTPUT:
We need to understand how different things work together to make the world a better place. 

* Economic growth (making money)
* Protecting the environment (keeping the planet clean)
* Fairness for everyone (social equity)

These things are connected. We need to make big c

## Evaluation (TODO)

In [30]:
def calculate_llm_similarity(original: str, simplified: str, api_key: str) -> float:
    """
    Uses LLM to evaluate semantic similarity (0.0 to 1.0) between original and simplified text.
    Handles paraphrasing better than TF-IDF.
    """
    system_prompt = """You are an expert evaluator. 
Compare the meaning of the Original Text and the Simplified Text.
Ignore any differences in complexity, style, or tone. Focus ONLY on whether the key information and meaning is preserved.
Even if the words are completely different (paraphrased), if the meaning is the same, score it high.

Rate the Semantic Similarity on a scale of 0.0 to 1.0, where:
1.0 = Perfect meaning preservation (key ideas are all present).
0.0 = Completely different meaning.

Output ONLY the float number (e.g. 0.95), nothing else."""

    user_prompt = f"Original Text:\n{original}\n\nSimplified Text:\n{simplified}"
    
    try:
        # Use low temp for consistency
        response = call_groq(system_prompt, user_prompt, api_key, temperature=0.0, max_tokens=10)
        # Clean up response to get just the number
        cleaned_response = response.strip().replace("'", "").replace("\"", "")
        return float(cleaned_response)
    except Exception as e:
        print(f"Error in eval: {e}")
        return 0.0

def evaluate_outputs(df: pd.DataFrame, api_key: str) -> pd.DataFrame:
    df_eval = df.copy()
    print("Calculating Semantic Similarity (LLM-based)...")
    df_eval['sim_a'] = df_eval.apply(lambda row: calculate_llm_similarity(row['original_text'], row['output_a'], api_key), axis=1)
    df_eval['sim_b'] = df_eval.apply(lambda row: calculate_llm_similarity(row['original_text'], row['output_b'], api_key), axis=1)
    return df_eval

# Run if comparison_df exists
if 'comparison_df' in locals():
    eval_df = evaluate_outputs(comparison_df, GROQ_API_KEY)
    
    print("\n" + "="*80)
    print("EVALUATION RESULTS (Semantic Similarity - LLM Judge)")
    print("="*80)
    print(eval_df[['filename', 'sim_a', 'sim_b']])
    print("-" * 80)
    print(f"Average Similarity Prompt A: {eval_df['sim_a'].mean():.4f}")
    print(f"Average Similarity Prompt B: {eval_df['sim_b'].mean():.4f}")

Calculating Semantic Similarity (LLM-based)...

EVALUATION RESULTS (Semantic Similarity - LLM Judge)
            filename  sim_a  sim_b
0    en_academic.txt   0.85   0.85
1  en_government.txt   0.85   0.85
2   en_insurance.txt   0.95   0.95
3       en_legal.txt   0.95   0.95
4     en_medical.txt   0.95   0.85
5   en_technical.txt   0.95   0.95
--------------------------------------------------------------------------------
Average Similarity Prompt A: 0.9167
Average Similarity Prompt B: 0.9000
